In [1]:
import dotenv
dotenv.load_dotenv()

False

In [2]:
from datasets import load_dataset
swebench = load_dataset('princeton-nlp/SWE-bench', split='dev')
swebench[0]

/Users/mark/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


{'repo': 'sqlfluff/sqlfluff',
 'instance_id': 'sqlfluff__sqlfluff-4764',
 'base_commit': 'a820c139ccbe6d1865d73c4a459945cd69899f8f',
 'patch': 'diff --git a/src/sqlfluff/cli/commands.py b/src/sqlfluff/cli/commands.py\n--- a/src/sqlfluff/cli/commands.py\n+++ b/src/sqlfluff/cli/commands.py\n@@ -44,6 +44,7 @@\n     dialect_selector,\n     dialect_readout,\n )\n+from sqlfluff.core.linter import LintingResult\n from sqlfluff.core.config import progress_bar_configuration\n \n from sqlfluff.core.enums import FormatType, Color\n@@ -691,12 +692,16 @@ def lint(\n         sys.exit(EXIT_SUCCESS)\n \n \n-def do_fixes(lnt, result, formatter=None, **kwargs):\n+def do_fixes(\n+    result: LintingResult, formatter: Optional[OutputStreamFormatter] = None, **kwargs\n+):\n     """Actually do the fixes."""\n-    click.echo("Persisting Changes...")\n+    if formatter and formatter.verbosity >= 0:\n+        click.echo("Persisting Changes...")\n     res = result.persist_changes(formatter=formatter, **kwargs)\

In [3]:
from langgraph.graph import StateGraph, add_messages
from typing import TypedDict, Annotated
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from vm import VirtualMachine
from agent_tools import create_tools

def init_repo_env(vm, repo_name: str, commit_hash: str)->None:
    repo_url = f"https://github.com/{repo_name}.git"
    vm.run_command(f"git clone {repo_url}")
    repo_dir = repo_name.split('/')[-1]
    vm.run_command(f"cd {repo_dir} && git checkout {commit_hash}")

def do_prediction(data):
    with VirtualMachine(image_name='swe_img:latest') as vm:
        init_repo_env(vm, data['repo'], data['base_commit'])
        vm.copy_file_to_vm('vm_tools.py', '/tools.py')

        llm = ChatOpenAI(model='gpt-4o-mini')
        tools = create_tools(vm)

        agent = create_react_agent(llm, tools=tools.values(), debug=True)
        
        final_state = agent.invoke(
            input={
                'messages': [
                    ('system', 'You are an autonomous programmer, and you are going to propose a pull request based on the problem below. You are given several tools to help you with the task. The requested repo has been cloned to the root directory. You can use the tools to view, edit, and create files, run Linux commands, and submit the patch.'),
                    ('user', f'Please submit a pull request based on the following problem:\nProblem: {data["problem_statement"]}\nRepo: {data["repo"]}\nHints: {data["hints_text"]}\nCreated at: {data["created_at"]}\nNow, start your work.\n'),
                ]
            },
            config={
                "recursion_limit": 50
            }
        )
    
    return final_state

In [4]:
result = do_prediction(swebench[0])
print(result)

[0:tasks] Starting step 0 with 1 task:
- __start__ -> {'messages': [('system',
               'You are an autonomous programmer, and you are going to propose '
               'a pull request based on the problem below. You are given '
               'several tools to help you with the task. The requested repo '
               'has been cloned to the root directory. You can use the tools '
               'to view, edit, and create files, run Linux commands, and '
               'submit the patch.'),
              ('user',
               'Please submit a pull request based on the following problem:\n'
               'Problem: Enable quiet mode/no-verbose in CLI for use in '
               'pre-commit hook\n'
               'There seems to be only an option to increase the level of '
               'verbosity when using SQLFluff '
               '[CLI](https://docs.sqlfluff.com/en/stable/cli.html), not to '
               'limit it further.\r\n'
               '\r\n'
               'It wo